### Importación de librerías

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import MinMaxScaler
mmscaler = MinMaxScaler()
from tqdm import tqdm
import re

import warnings
warnings.filterwarnings("ignore")

### Lectura del archivo excel con la información de los clientes

In [ ]:
df_tabla_final = pd.read_excel("../archivos_entrada_script/tabla_final.xlsx", sheet_name="Hoja2", dtype={"CODIGOCLIE":str})
df_tabla_final["CODIGOCLIE"] = df_tabla_final["CODIGOCLIE"].astype("string")
df_tabla_final = df_tabla_final.iloc[:,2:6]
df_tabla_final

In [9]:
clientes_unicos = df_tabla_final["CODIGOCLIE"].unique()
print(clientes_unicos)

<StringArray>
['1401867926', '1490000744', '1490000688', '1490001175', '1490000515',
 '1401337167', '1490002225', '1401582655', '1490001247', '1401282968',
 ...
 '1401229975', '1401954998', '1490000719', '1401940609', '1410026628',
 '1410020880', '1490000693', '1490001644', '1410062892', '1490002155']
Length: 214, dtype: string


In [2]:
def intentar_abrir_archivo_datos(path_archivo):

    df_archivo_telem_tab = pd.DataFrame([1,2,3], columns=["prueba"])
    df_archivo_telem_pc = pd.DataFrame([1,2,3], columns=["prueba"])
    df_archivo_telem_c = pd.DataFrame([1,2,3], columns=["prueba"])

    # Intentar leer el archivo con separador 'tab'
    try:
        #print("Leyendo con tab")
        df_archivo_telem_tab = pd.read_csv(path_archivo,
                                            sep="\t",
                                            #decimal=",",
                                            skiprows=11,
                                            #na_values="N/D",
                                            encoding="utf-16",
                                            #on_bad_lines="skip",
                                            encoding_errors="ignore"
                                            ) 
    except:
        pass

    # Intentar leer el archivo con separador 'punto y coma'
    try:
        #print("Leyendo con ;")
        df_archivo_telem_pc = pd.read_csv(path_archivo,
                                            sep=";",
                                            #decimal=",",
                                            skiprows=11,
                                            #na_values="N/D",
                                            #encoding="utf-16",
                                            #on_bad_lines="skip",
                                            encoding_errors="ignore" 
                                            )

    except:
        pass

    # Intentar leer el archivo con separador 'coma'
    try:
        #print("Leyendo con ,")
        df_archivo_telem_c = pd.read_csv(path_archivo,
                                            sep=",",
                                            #decimal=",",
                                            skiprows=11,
                                            #na_values="N/D",
                                            #encoding="utf-16",
                                            #on_bad_lines="skip",
                                            encoding_errors="ignore" 
                                            )

    except:
        pass



    dfs = [df_archivo_telem_tab, df_archivo_telem_pc, df_archivo_telem_c]
    
    # Elegir el primer DF válido (más de 1 columna) o el último como fallback
    for df in dfs:
        if df.shape[1] > 1:
            return df

    return dfs[-1]

### 1. Extracción de los archivos de cada grupo de clientes

#### 1.1 Extracción de los archivos con las mediciones mensuales del primer grupo de clientes

Estos clientes están clasificados por CUEN, y el formato de sus archivos varían, el procedimiento a seguir será:
1. Iterar sobre cada código de cliente
2. Buscar sus archivos de datos mensuales
3. Unificar todos en un solo archivo
4. El identificador a usar es el código de cliente

In [3]:
mediciones_clientes_g1 = r"../data/mediciones_por_mes_g1"
archivos_mediciones_g1 = list(os.scandir(mediciones_clientes_g1))
columnas_extraer_g1 = ["Fecha","Demanda activa DEL","Demanda reactiva DEL"]

In [4]:
clientes_unicos_g1 = set()

for medicion in archivos_mediciones_g1:
    cliente = medicion.name.split('-')[1]
    clientes_unicos_g1.add(cliente)

print(f"Clientes único encontrados en la carpeta de mediciones del grupo uno: {len(clientes_unicos_g1)}")

Clientes único encontrados en la carpeta de mediciones del grupo uno: 313


In [5]:
# Diccionario para almacenar los clientes con sus respectivos datos
dict_dfs_clientes_g1 = {}

# Iterar sobre cada cliente
for cliente in tqdm(clientes_unicos_g1, desc="Procesando clientes del grupo 01"):
    datos_cliente = []
    # Iterar sobre cada archivo de medicion del grupo 01
    for medicion in archivos_mediciones_g1:
        if cliente == medicion.name.split("-")[1]:
            df_cliente = intentar_abrir_archivo_datos(f"{mediciones_clientes_g1}/{medicion.name}")
            datos_cliente.extend(df_cliente[columnas_extraer_g1].values)

    # Convertir a DataFrame los datos concatenados
    df_datos_anual_cliente = pd.DataFrame(datos_cliente, columns=columnas_extraer_g1)
    
    # Almacenar en el diccionario (Clave->Cliente   Valor->DataFrame)
    dict_dfs_clientes_g1[cliente]=df_datos_anual_cliente
    
    # Eliminar dataframe concatenado para liberar memoria
    del df_datos_anual_cliente
    #df_datos_anual_cliente.to_csv(f"mediciones_por_anio/g1_perfil_carga_anual-{cliente}-2023.csv", index=False)

Procesando clientes del grupo 01: 100%|██████████| 313/313 [01:48<00:00,  2.89it/s]


#### 1.2 Extracción de los archivos con las mediciones mensuales del segundo grupo de clientes

De estos clientes tenemos carpetas con sus mediciones por mes, no existe tabla de excel inicial, se procederá a realizar lo siguiente:
1. Iterar sobre cada carpeta (cliente)
2. Obtener los datos de sus 12 meses
3. Unificar en un único archivo anual
4. Se usará el nombre del cliente como identificador

In [6]:
mediciones_clientes_g2 = "../data/mediciones_por_mes_g2"
archivos_mediciones_g2 = list(os.scandir(mediciones_clientes_g2))
columnas_extraer_g2 = ["Fecha", "AS (kWh)"]

In [7]:
print(f"Clientes único encontrados en la carpeta de mediciones del grupo uno: {len(archivos_mediciones_g2)}")

Clientes único encontrados en la carpeta de mediciones del grupo uno: 75


In [8]:
# Diccionario para almacenar los clientes con sus datos
dict_dfs_clientes_g2 = {}

# Iterar sobre cada cliente
for archivos_cliente in tqdm(archivos_mediciones_g2, desc="Procesando clientes del grupo 02"):
    nombre_cli = archivos_cliente.name.strip()
    df_concat = pd.DataFrame()

    # Obtener los archivos de las mediciones mensuales del cliente
    mediciones_mensuales_cliente = os.scandir(rf"{mediciones_clientes_g2}/{nombre_cli}")

    for medicion in mediciones_mensuales_cliente:
        medicion_mensual = pd.read_csv(rf"{mediciones_clientes_g2}/{nombre_cli}/{medicion.name}", sep=";", skiprows=2, encoding='ISO-8859-1')
        medicion_mensual = medicion_mensual[columnas_extraer_g2]
        df_concat = pd.concat([df_concat, medicion_mensual])

    # Almacenar en el diccionario (Clave->Cliente   Valor->DataFrame)
    dict_dfs_clientes_g2[nombre_cli] = df_concat
    
    # Eliminar dataframe concatenado para liberar memoria
    del df_concat
    #df_concat.to_csv(rf"mediciones_por_anio/g2_perfil_carga_anual-{nombre_cli}-2023.csv", index=False)

Procesando clientes del grupo 02: 100%|██████████| 75/75 [00:18<00:00,  4.00it/s]


### 2. Preprocesamiento de los datos

Ahora tenemos todos los datos unificados anualmente por cada cliente, tenemos que limpiar y preprocesar, realizar las siguientes tareas:
1. Calcular la potencia aparente (resultado de aplicar teorema pitágoras sobre potencia activa y reactiva)
2. Separar la columna 'fecha' en dos columnas 'fecha' y 'hora', fecha va a tener formato 'año/mes/dia' y hora el formato 'hh:mm'
3. Excluir aquellos registros que correspondan a fechas de sábado, domingo o días de feriado nacional
4. Normalizar los datos para que todos estén en la misma escala

In [9]:
feriados_nacionales = ["2/7/2023", "20/2/2023", "21/2/2023", "7/4/2023", "1/5/2023", \
                       "26/5/2023", "11/8/2023", "9/10/2023", "2/10/2023", "3/10/2023", "25/12/2023"]
feriados_nacionales = pd.to_datetime(feriados_nacionales, format='%d/%m/%Y')

dict_meses = {"01": "Enero",
              "02": "Febrero",
              "03": "Marzo",
              "04": "Abril",
              "05": "Mayo",
              "06": "Junio",
              "07": "Julio",
              "08": "Agosto",
              "09": "Septiembre",
              "10": "Octubre",
              "11": "Noviembre",
              "12": "Diciembre"}

print(list(dict_meses.keys()))
print(feriados_nacionales)

['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
DatetimeIndex(['2023-07-02', '2023-02-20', '2023-02-21', '2023-04-07',
               '2023-05-01', '2023-05-26', '2023-08-11', '2023-10-09',
               '2023-10-02', '2023-10-03', '2023-12-25'],
              dtype='datetime64[ns]', freq=None)


In [10]:
def fecha_formato_unico(fecha_str):

    # Poner separador único el '/' y año únicamente 2023
    fecha_str = fecha_str.replace('-','/').replace('2024','2023')

    if len(fecha_str.split('/')[0]) == 4: # Cuando el formato es año/mes/día
        return fecha_str
    elif len(fecha_str.split('/')[0]) != 4: # Cuando el formato es día/mes/año
        seps = fecha_str.split('/')
        return f"{seps[-1]}/{seps[1]}/{seps[0]}"
    
def hora_formato_unico(hora_str):
    if pd.isna(hora_str):
        return np.nan
    match = re.match(r'^(\d{1,2}:\d{2})(:\d{2})?$', hora_str.strip())
    return match.group(1) if match else np.nan
    

def recortar_valores_atipicos(df, columna_valor):
    # Calcular los cuartiles y el IQR
    Q1 = df[columna_valor].quantile(0.25)
    Q3 = df[columna_valor].quantile(0.75)
    IQR = Q3 - Q1

    # Definir los límites superior e inferior
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR

    # Aplicar el recorte
    df[columna_valor] = df[columna_valor].clip(lower=limite_inferior, upper=limite_superior)

2.1 - Transformación columna fecha archivos anuales grupo 01

In [11]:
dict_dfs_procesados_g1 = {}

for cliente, df_archivo_g1 in dict_dfs_clientes_g1.items():

    # Transformar columna fecha a cadena
    df_archivo_g1["Fecha"] = df_archivo_g1["Fecha"].astype("string")

    # Eliminar registros con valores nulos en columna 'Fecha'
    df_archivo_g1 = df_archivo_g1.dropna(subset="Fecha")

    # Separar para obtener columna Hora
    df_archivo_g1["Hora"] = df_archivo_g1["Fecha"].apply(lambda x: x.split()[1].strip())

    # Separar para obtener columna Fecha
    df_archivo_g1["Fecha"] = df_archivo_g1["Fecha"].apply(lambda x: x.split()[0].strip())

    # Transformar la columna Fecha a un formato único
    df_archivo_g1["Fecha"] = df_archivo_g1["Fecha"].apply(fecha_formato_unico)
    
    # Transformar la columna Hora a un formato único hh:mm
    df_archivo_g1["Hora"] = df_archivo_g1["Hora"].apply(hora_formato_unico)

    # Debido a que a veces se ponen datos del 2024 para reemplazar los faltantes del 2023
    # debemos descartar la fecha 29 de febrero, pues en 2023 no existe
    df_archivo_g1 = df_archivo_g1[df_archivo_g1["Fecha"] != "2023/02/29"]
    
    # Eliminar duplicados y conservar el original del 2023
    df_archivo_g1["Fecha-Hora"] = df_archivo_g1["Fecha"] + " " + df_archivo_g1["Hora"]
    df_archivo_g1 = df_archivo_g1.drop_duplicates(subset="Fecha-Hora", keep="first")

    # Transformar columna Fecha a datetime
    df_archivo_g1["Fecha"] = pd.to_datetime(df_archivo_g1["Fecha"], format='%Y/%m/%d')

    # Obtener la potencia aparente
    df_archivo_g1["Potencia_aparente"] = np.sqrt((df_archivo_g1["Demanda activa DEL"]**2) + (df_archivo_g1["Demanda reactiva DEL"]**2))

    # Eliminar días feriados y días de fin de semana
    df_archivo_g1 = df_archivo_g1[~df_archivo_g1['Fecha'].isin(feriados_nacionales) & ~df_archivo_g1['Fecha'].dt.weekday.isin([5, 6])]

    # Interpolar valores nulos usando una función polinomial
    df_archivo_g1["Potencia_aparente"] = df_archivo_g1["Potencia_aparente"].interpolate(method='cubicspline')

    # Conservar solo las columnas de interés
    df_archivo_g1 = df_archivo_g1[["Fecha", "Hora", "Potencia_aparente"]]

    # Escalar las mediciones
    df_archivo_g1["Potencia_aparente_escalada"] = mmscaler.fit_transform(df_archivo_g1[["Potencia_aparente"]])

    # Guardar en un nuevo diccionario los datos procesados
    dict_dfs_procesados_g1[cliente] = df_archivo_g1
    
    # Liberar memoria
    del df_archivo_g1
    
del dict_dfs_clientes_g1

2.2 - Transformación columna fecha archivos anuales grupo 02

In [12]:
dict_dfs_procesados_g2 = {}

for cliente, df_archivo_g2 in dict_dfs_clientes_g2.items():

    # Transformar columna fecha a cadena
    df_archivo_g2["Fecha"] = df_archivo_g2["Fecha"].astype("string")

    # Eliminar registros que contienen el total
    df_archivo_g2 = df_archivo_g2[~df_archivo_g2["Fecha"].str.contains("Total")]

    # Eliminar registros con valores nulos en columna 'Fecha'
    df_archivo_g2 = df_archivo_g2.dropna(subset="Fecha")

    # Separar para obtener columna Hora
    df_archivo_g2["Hora"] = df_archivo_g2["Fecha"].apply(lambda x: x.split()[1].strip())

    # Separar para obtener columna Fecha
    df_archivo_g2["Fecha"] = df_archivo_g2["Fecha"].apply(lambda x: x.split()[0].strip())

    # Eliminar duplicados y conservar el original del 2023
    df_archivo_g2["Fecha-Hora"] = df_archivo_g2["Fecha"] + " " + df_archivo_g2["Hora"]
    df_archivo_g2["Fecha-Hora"] = pd.to_datetime(df_archivo_g2["Fecha-Hora"], yearfirst=True)
    df_archivo_g2 = df_archivo_g2.drop_duplicates(subset="Fecha-Hora", keep="first")

    # Restar un timedelta de 15 a todas las fechas (Solo en este caso por que la ultima fecha)
    # se pasa al siguiente mes
    df_archivo_g2["Fecha-Hora"] = df_archivo_g2["Fecha-Hora"] - pd.Timedelta(minutes=15)

    # Separar nuevamente para obtener columna Hora
    df_archivo_g2["Hora"] = df_archivo_g2["Fecha-Hora"].astype("string").apply(lambda x: x.split()[1][:-3].strip())

    # Separar nuevamente para obtener columna Fecha
    df_archivo_g2["Fecha"] = df_archivo_g2["Fecha-Hora"].astype("string").apply(lambda x: x.split()[0].strip())

    # Transformar la columna Fecha a un formato único
    df_archivo_g2["Fecha"] = df_archivo_g2["Fecha"].apply(fecha_formato_unico)

    # Debido a que a veces se ponen datos del 2024 para reemplazar los faltantes del 2023
    # debemos descartar la fecha 29 de febrero, pues en 2023 no existe
    df_archivo_g2 = df_archivo_g2[df_archivo_g2["Fecha"] != "2023/02/29"]

    # Transformar columna Fecha a datetime
    df_archivo_g2["Fecha"] = pd.to_datetime(df_archivo_g2["Fecha"], format='%Y/%m/%d')

    # Limpiar la columna 'SE (KVah)'
    df_archivo_g2["AS (kWh)"] = df_archivo_g2["AS (kWh)"].astype("string").str.replace(",", "").replace('"','')
    df_archivo_g2["AS (kWh)"] = df_archivo_g2["AS (kWh)"].astype("float")

    # Obtener la potencia aparente
    df_archivo_g2["Potencia_aparente"] = df_archivo_g2["AS (kWh)"] * 4

    # Eliminar días feriados y días de fin de semana
    df_archivo_g2 = df_archivo_g2[~df_archivo_g2['Fecha'].isin(feriados_nacionales) & ~df_archivo_g2['Fecha'].dt.weekday.isin([5, 6])]

    # Interpolar valores nulos usando una función polinomial
    df_archivo_g2["Potencia_aparente"] = df_archivo_g2["Potencia_aparente"].interpolate(method='cubicspline')

    # Conservar solo las columnas de interés
    df_archivo_g2 = df_archivo_g2[["Fecha", "Hora", "Potencia_aparente"]]

    # Escalar las mediciones
    df_archivo_g2["Potencia_aparente_escalada"] = mmscaler.fit_transform(df_archivo_g2[["Potencia_aparente"]])

    # Guardar en un nuevo diccionario los datos procesados
    dict_dfs_procesados_g2[cliente] = df_archivo_g2
    
    # Liberar memoria
    del df_archivo_g2
    
del dict_dfs_clientes_g2

### 3. Generación de los entregables

In [14]:
def obtener_coords_curva_tipo(df):

    # Agrupar por hora y aplicar mediana
    df_grouped = df.groupby("Hora")["Potencia_aparente_escalada"].apply(np.median).sort_index(ascending=True).reset_index(drop=False)

    # Retornar el array con los 96 valores de demanda
    return df_grouped

def obtener_coords_dia_demanda_max(df):

    # Obtener el máximo valor de potencia aparente
    max_potencia = df['Potencia_aparente_escalada'].max()

    # Encontrar la fecha correspondiente a la máxima potencia aparente
    fecha_max_potencia = df[df['Potencia_aparente_escalada'] == max_potencia]['Fecha'].iloc[0]

    # Filtrar los registros correspondientes a esa fecha
    df_max_fecha = df[df['Fecha'] == fecha_max_potencia]

    # Ordenar el resultado de manera ascendente por 'Hora'
    df_max_fecha = df_max_fecha.sort_values(by="Hora", ascending=True)

    return fecha_max_potencia, df_max_fecha[["Hora","Potencia_aparente_escalada"]]

In [15]:
def graficar_curva_tipo(df, cod_cli, path):

    # Generar el gráfico de la curva tipo
    _ = plt.figure(figsize=(16, 6))
    _ = plt.plot(df["Hora"], df["Potencia_aparente_escalada"], marker='o', color='b', linestyle='-', label='Potencia Aparente Escalada')
    _ = plt.title(f'Curva tipo cliente {cod_cli}')
    _ = plt.xlabel('Hora')
    _ = plt.ylabel('Potencia Aparente Escalada')
    _ = plt.grid(True)

    # Rotar etiquetas para que no se vea acumulado el eje X
    _ = plt.xticks(df["Hora"].values[::2], rotation=45)

    # Para que no se distorsione la dimenisión del eje Y
    _ = plt.yticks(np.arange(0, 1.1, 0.1))

    # Evitar recortes en las etiquetas
    _ = plt.tight_layout()

    # Guardar la gráfica en un directorio
    _ = plt.savefig(f"{path}/curva_tipo_{cod_cli}.png", format='png')  # Puedes cambiar el formato a 'jpg', 'pdf', etc.
    _ = plt.savefig(f"../outputs/curvas_tipo/curva_tipo_{cod_cli}.png", format='png')  # Puedes cambiar el formato a 'jpg', 'pdf', etc.
    
    # Cerrar la figura después de guardarla para liberar recursos
    _ = plt.close()  

def graficar_dia_max_demanda(df, cod_cli, path, fecha):

    # Graficar la potencia aparente escalada a lo largo del día
    _ = plt.figure(figsize=(16, 6))
    _ = plt.plot(df["Hora"], df["Potencia_aparente_escalada"], marker='o', color='r', linestyle='-', label=f'Potencia Aparente Escalada')
    _ = plt.title(f'Curva del día de demanda máxima {fecha} para cliente {cod_cli}')
    _ = plt.xlabel('Hora')
    _ = plt.ylabel('Potencia Aparente Escalada')
    _ = plt.grid(True)
    _ = plt.legend()

    # Rotar etiquetas para que no se vea acumulado el eje X
    _ = plt.xticks(df["Hora"].values[::2], rotation=45)

    # Para que no se distorsione la dimenisión del eje Y
    _ = plt.yticks(np.arange(0, 1.1, 0.1))

    # Evitar recortes en las etiquetas
    _ = plt.tight_layout()

    # Guardar la gráfica en un archivo (por ejemplo, como archivo PNG)
    _ = plt.savefig(f"{path}/curva_dia_demanda_max_{cod_cli}.png", format='png')  # Puedes cambiar el formato a 'jpg', 'pdf', etc.
    
    # Cerrar la figura después de guardarla para liberar recursos
    _ = plt.close()

In [16]:
def agrupar_horas(df, columna_hora="Hora", columna_valor="Potencia_aparente_escalada"):

    # Convertir la columna Hora a tipo datetime
    df["Hora"] = pd.to_datetime(df[columna_hora], format="%H:%M")

    # Agrupar por hora y tomar el punto medio (hh:30)
    df["Hora"] = df["Hora"].dt.floor("H") + pd.Timedelta(minutes=30)

    # Recortar a HH:MM
    df["Hora"] = df["Hora"].astype("string").apply(lambda x: x.split()[1][:-3])

    # Agrupar por la nueva columna de hora y calcular el promedio de los valores
    return df.groupby("Hora")[columna_valor].apply(np.mean).reset_index()

In [17]:
def agrupar_30_min(df, columna_hora="Hora", columna_valor="Potencia_aparente_escalada"):

    # Convertir la columna Hora a tipo datetime
    df["Hora"] = pd.to_datetime(df[columna_hora], format="%H:%M")

    # Redondear hacia arriba al final del intervalo de 30 minutos
    df["Hora"] = df["Hora"] + pd.Timedelta(minutes=30)
    df["Hora"] = df["Hora"].dt.floor("H") + (df["Hora"].dt.minute // 30) * pd.Timedelta(minutes=30)

    # Recortar a HH:MM
    df["Hora"] = df["Hora"].astype("string").apply(lambda x: x.split()[1][:-3])

    # Agrupar por la nueva columna de hora y calcular el promedio de los valores
    return df.groupby("Hora")[columna_valor].apply(np.mean).reset_index()

### Proceso de agregación y obtención de las curvas

In [18]:
path_entregables = "../outputs"

dict_todos_los_clientes = dict_dfs_procesados_g1 | dict_dfs_procesados_g2

In [19]:
carpeta_entregables = 'entregables_por_cliente_v1'
generar_entregables = False

In [20]:
#del dict_dfs_procesados_g1
#del dict_dfs_procesados_g2


registros_curvas_todas = []

for cliente, df_medicion_anual in dict_todos_los_clientes.items():

    # Lista para almacenar todos los valores de la curva tipo del cliente
    registros_curva_cliente = []

    # Directorio entregables cliente
    dir_entregables_cli = fr"{path_entregables}/{carpeta_entregables}/{cliente}"

    # Crear el directorio para los entregables (Si no existe)
    if not os.path.exists(dir_entregables_cli):
        os.makedirs(os.path.normpath(dir_entregables_cli))

    # Generar el archivo con los datos de la curva tipo
    df_curva_tipo = obtener_coords_curva_tipo(df_medicion_anual)
    #df_curva_tipo = agrupar_30_min(df_curva_tipo)
    
    # Guardar los datos en una lista
    registros_curva_cliente.append(cliente)
    for valor in df_curva_tipo["Potencia_aparente_escalada"].values:
        registros_curva_cliente.append(valor)
    
    if generar_entregables == True:
        # Generar un .csv con los datos de la curva tipo
        df_curva_tipo.to_csv(f"{dir_entregables_cli}/datos_curva_tipo_{cliente}.csv", index=False)

        # Generar el archivo con los datos de la curva del día que hubo la demanda máxima
        fecha_max_dem, df_curva_dia_dem_max = obtener_coords_dia_demanda_max(df_medicion_anual)
        df_curva_dia_dem_max = df_curva_dia_dem_max.sort_values(by="Hora", ascending=True)
        
        # Generar un .csv con los datos de la curva del día de demanda máximo
        df_curva_dia_dem_max.to_csv(f"{dir_entregables_cli}/datos_curva_dia_demanda_max.csv", index=False)

        # Generar la gráfica de la curva tipo
        graficar_curva_tipo(df_curva_tipo, cliente, dir_entregables_cli)

        # Generar la gráfica de la curva del día de demanda máxima
        graficar_dia_max_demanda(df_curva_dia_dem_max, cliente, dir_entregables_cli, str(fecha_max_dem).split()[0])

        # Generar un archivo plano con la demanda máximo y mínima
        open(rf'{dir_entregables_cli}/Potencia_max_min.txt', 'w')\
            .write(f'Pot_aparente_max: {df_medicion_anual["Potencia_aparente"].max()}\nPot_aparente_min: {df_medicion_anual["Potencia_aparente"].min()}')

    # Guardar la lista con los registros de un cliente en otra lista
    registros_curvas_todas.append(registros_curva_cliente)

columnas_df_todas_las_curvas = ["Cliente"]
columnas_df_todas_las_curvas.extend(df_curva_tipo["Hora"].values)

### 4. Cargar los datos a MongoDB (únicamente los de curvas tipo)

In [21]:
del dict_todos_los_clientes
del dict_dfs_procesados_g1
del dict_dfs_procesados_g2

In [22]:
df_registros_todas_las_curvas = pd.DataFrame(registros_curvas_todas)
df_registros_todas_las_curvas.columns = columnas_df_todas_las_curvas
df_registros_todas_las_curvas

,Cliente,00:00,00:15,00:30,00:45,01:00,01:15,01:30,01:45,02:00,...,21:30,21:45,22:00,22:15,22:30,22:45,23:00,23:15,23:30,23:45
0,90001271,0.489612,0.493995,0.492886,0.499559,0.494802,0.492869,0.494802,0.490005,0.493607,...,0.524552,0.527898,0.515291,0.498844,0.497775,0.500199,0.508555,0.510100,0.508617,0.507770
1,90000401,0.237491,0.230294,0.237011,0.219739,0.202467,0.194791,0.199588,0.187114,0.193831,...,0.282125,0.273474,0.266600,0.267565,0.271555,0.279007,0.263879,0.276353,0.265730,0.262079
2,730664,0.004355,0.003920,0.003484,0.003484,0.003049,0.003484,0.003049,0.003049,0.003049,...,0.021123,0.021125,0.018728,0.016115,0.011106,0.008928,0.007840,0.006968,0.006533,0.006097
3,90002524,0.677454,0.677856,0.674504,0.678018,0.676759,0.671495,0.680169,0.680929,0.677969,...,0.585613,0.591873,0.617582,0.671971,0.667079,0.675583,0.683747,0.672458,0.673943,0.682512
4,90001692,0.145248,0.150058,0.148129,0.145812,0.145816,0.141964,0.141193,0.141584,0.137735,...,0.194026,0.190068,0.184080,0.178572,0.169847,0.170024,0.170336,0.165148,0.163133,0.161903
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383,SIGMAPLAST,0.720672,0.723474,0.731963,0.731715,0.730960,0.732802,0.723777,0.716437,0.719621,...,0.746858,0.754012,0.754291,0.757646,0.755949,0.755628,0.759778,0.754183,0.754331,0.762028
384,SINTOFIL,0.610181,0.607328,0.608991,0.606137,0.611956,0.606191,0.601364,0.600915,0.604288,...,0.677249,0.666129,0.665000,0.669310,0.650399,0.653535,0.654690,0.667381,0.664152,0.671782
385,SOCIEDAD INDUSTRIAL RELI CYRANO,0.514915,0.547054,0.585794,0.610724,0.638181,0.617241,0.576222,0.520726,0.484861,...,0.271258,0.333978,0.380271,0.425718,0.485140,0.508154,0.535512,0.547810,0.520870,0.522786
386,TEXTILES TEXSA,0.740500,0.726816,0.741630,0.747327,0.742125,0.738670,0.738810,0.727922,0.709047,...,0.734962,0.741010,0.740593,0.739784,0.741396,0.733946,0.750470,0.750396,0.750901,0.751180


In [23]:
### Verificar valores nulos
df_registros_todas_las_curvas[df_registros_todas_las_curvas.isna().any(axis=1)]

,Cliente,00:00,00:15,00:30,00:45,01:00,01:15,01:30,01:45,02:00,...,21:30,21:45,22:00,22:15,22:30,22:45,23:00,23:15,23:30,23:45


In [120]:
### Excluir filas con valores nulos o filas duplicadas
df_registros_todas_las_curvas = df_registros_todas_las_curvas.drop_duplicates(subset="Cliente")

In [24]:
df_registros_todas_las_curvas.shape

(388, 97)

In [25]:
from dotenv import load_dotenv
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
from pymongo import MongoClient

def obtener_cliente_db():
    # Cargar las variables del archivo .env
    load_dotenv()

    # Obtener las credenciales y el cluster de MongoDB
    username = os.getenv("DB_USER_T")
    password = os.getenv("DB_PASS_T")
    cluster = os.getenv("DB_CLUSTER_T")

    # Construir la uri con las credenciales
    uri = f"mongodb+srv://{username}:{password}@{cluster.lower()}.gypwd.mongodb.net/?retryWrites=true&w=majority&appName={cluster}"

    # Crear un cliente y conectarlo al servidor
    client = MongoClient(uri, 
                        server_api=ServerApi('1'),
                        connectTimeoutMS=60000,
                        socketTimeoutMS=60000,
                        serverSelectionTimeoutMS=60000,
                        tls=True)

    # Si el cliente existe, retornarlo
    if client is not None:
        return client

In [26]:
# Obtener cliente para conectar a la db
db_cliente = obtener_cliente_db()

# Convertir el dataframe a diccionario
datos_insertar = df_registros_todas_las_curvas.to_dict(orient='records')

# Eliminar cualquier documento existente en la colección
db_cliente.CurvasTipo.CurvasTipoAnualesV1.delete_many({})

# Creación de índice para el campo de la fecha
db_cliente.CurvasTipo.CurvasTipoAnualesV1.create_index([("Cliente", 1)])

# Insertar en la base de datos
db_cliente.CurvasTipo.CurvasTipoAnualesV1.insert_many(datos_insertar)

InsertManyResult([ObjectId('692bbad153645496bdadb874'), ObjectId('692bbad153645496bdadb875'), ObjectId('692bbad153645496bdadb876'), ObjectId('692bbad153645496bdadb877'), ObjectId('692bbad153645496bdadb878'), ObjectId('692bbad153645496bdadb879'), ObjectId('692bbad153645496bdadb87a'), ObjectId('692bbad153645496bdadb87b'), ObjectId('692bbad153645496bdadb87c'), ObjectId('692bbad153645496bdadb87d'), ObjectId('692bbad153645496bdadb87e'), ObjectId('692bbad153645496bdadb87f'), ObjectId('692bbad153645496bdadb880'), ObjectId('692bbad153645496bdadb881'), ObjectId('692bbad153645496bdadb882'), ObjectId('692bbad153645496bdadb883'), ObjectId('692bbad153645496bdadb884'), ObjectId('692bbad153645496bdadb885'), ObjectId('692bbad153645496bdadb886'), ObjectId('692bbad153645496bdadb887'), ObjectId('692bbad153645496bdadb888'), ObjectId('692bbad153645496bdadb889'), ObjectId('692bbad153645496bdadb88a'), ObjectId('692bbad153645496bdadb88b'), ObjectId('692bbad153645496bdadb88c'), ObjectId('692bbad153645496bdadb8